In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.path import Path
import seaborn as sns
import xarray as xr
import math

sys.path.append('..//')
from utils import translate_grid_to_origin, get_grid_angle, rotate_grid

In [ ]:
suffix_particle_files = 'diff_2Cs'
particle_folders =[rf"~//parcels_toolbox//02_output//artful_01_{suffix_particle_files}.zarr",
                  rf"~//parcels_toolbox//02_output//artful_02_{suffix_particle_files}.zarr",
                  rf"~//parcels_toolbox//02_output//artful_03_{suffix_particle_files}.zarr"]

plot_dates = [np.datetime64('2023-09-12 10:00:00'),
              np.datetime64('2023-11-08 11:00:00'),
              np.datetime64('2024-05-01 10:00:00')]

measure_file_paths = [ r"..//coord_sampling//measures_20230912.txt",
                       r"..//coord_sampling//measures_20231108.txt",
                       r"..//coord_sampling//measures_20240501.txt"]

In [ ]:
rotate = True

i_scenario = 2

nb_days_to_consider = 3
search_radius = 300 # [m]

particle_folder = particle_folders[i_scenario]
plot_date = plot_dates[i_scenario]
measure_file_path = measure_file_paths[i_scenario]

figure_output_folder_path = r'.//figures//age'
csv_output_folder_path = r'.//csv'

os.makedirs(figure_output_folder_path, exist_ok=True)
os.makedirs(csv_output_folder_path, exist_ok=True)

In [ ]:
from zoneinfo import ZoneInfo 
dt_naive = plot_date.astype('datetime64').astype(datetime)
dt_utc = dt_naive.replace(tzinfo=ZoneInfo("UTC"))
dt_bern = dt_utc.astimezone(ZoneInfo("Europe/Zurich"))


In [ ]:
str_Bern_plot_date = dt_bern.strftime('%Y-%m-%d %H:%M:%S %Z')
print(str_Bern_plot_date)

# Get particles

In [ ]:
xr_particules = xr.open_zarr(particle_folder) # obs= time index (0=the moment the particule is seeded), trajectory= particle

In [ ]:
arr_time = xr_particules.time.values.flatten()
df_time = pd.Series(arr_time)
df_time = df_time.dropna().drop_duplicates()

In [ ]:
df_time

In [ ]:
xr_particules['age'] = (xr_particules['time'] - xr_particules['time'].isel(obs=0)).dt.total_seconds() / 86400

### Select plot date 

In [ ]:
snapshot = xr_particules.where(xr_particules['time']==plot_date)
mask_snapshot = ~np.isnan(snapshot.lon.values)

In [ ]:
x_part_full, y_part_full, z_part_full, age_part_full = snapshot["lon"].values[mask_snapshot], snapshot["lat"].values[mask_snapshot], snapshot["z"].values[mask_snapshot], snapshot["age"].values[mask_snapshot]

In [ ]:
x_part = x_part_full[age_part_full<nb_days_to_consider]
y_part = y_part_full[age_part_full<nb_days_to_consider]
z_part = z_part_full[age_part_full<nb_days_to_consider]
age_part = age_part_full[age_part_full<nb_days_to_consider]

In [ ]:
plt.scatter(x_part, y_part)

## Convert particles coordinates to CH1903

In [ ]:
# point of origin in swiss grid 
X0_SG = 500000
Y0_SG = 116500

# second point for tilted grid
X1_SG = 563000
Y1_SG = 138700

angle = get_grid_angle(X0_SG, Y0_SG, X1_SG, Y1_SG)

In [ ]:
# XY on swiss grid 
x_sg = x_part + X0_SG
y_sg = y_part + Y0_SG

In [ ]:
if rotate:
    x_part, y_part = rotate_grid(x_sg, y_sg, X0_SG, Y0_SG, angle)

### Get location of measure points

In [ ]:
meas_coordinates = pd.read_csv(measure_file_path, delimiter=';', dtype=None, names=["label", "x", "y", "depths"], encoding='utf-8')

### Select particles close to the measure points

In [ ]:
def select_particules_from_distance(x_particules, y_particules, point_location, max_distance_from_point):
    # Calculate distances using vectorized operations
    distances = np.sqrt((x_particules - point_location['x'])**2 + (y_particules - point_location['y'])**2)
    
    # Select indices where distance is less than max_distance_from_location
    sel_i_particules = np.where(distances < max_distance_from_point)[0]
    
    # Convert to list if needed
    return list(sel_i_particules)

In [ ]:
i_sel_part_per_point = []
for index, row in meas_coordinates.iterrows():
    i_selected_particules = select_particules_from_distance(x_part, y_part, row, search_radius)
    i_sel_part_per_point.append(i_selected_particules)

### Concentration at location

In [ ]:
conc_init = 2000 * 50 / search_radius # particules / cellule

In [ ]:
concentrations = []
for points in i_sel_part_per_point:
    concentrations.append(len(points) / conc_init)

In [ ]:
meas_coordinates['simulated_concentration'] = concentrations
meas_coordinates.to_csv(os.path.join(csv_output_folder_path, rf'Artful0{i_scenario+1}_{suffix_particle_files}_concentrations_r{search_radius}m_{nb_days_to_consider}days.csv'))

### Age of selected particles

In [ ]:
for i_point in range(len(i_sel_part_per_point)):
    plt.close('all')
    fig = plt.figure(figsize=(4, 3))
    sns.kdeplot(age_part[i_sel_part_per_point[i_point]], color='blue')
    
    plt.text(0.98, 0.98, f'{str_Bern_plot_date}', transform=plt.gca().transAxes, ha='right', va='top')
    plt.text(0.98, 0.90, f'Concentration = {concentrations[i_point]}', transform=plt.gca().transAxes, ha='right', va='top')
    plt.xlabel('Age [days]')
    plt.ylabel('Density')
    plt.tight_layout()
    fig.savefig(os.path.join(figure_output_folder_path, rf'Artful0{i_scenario+1}_{suffix_particle_files}_point{i_point}_r{search_radius}m_{nb_days_to_consider}days.png'))